# IMPORTS

In [1]:
from collections import *
from functools import *
from itertools import *
import operator
import sys
import re
import math
import string
import bisect
import parse
import matplotlib.pyplot as plt
import numpy as np
import heapq
from matplotlib.path import Path
import ast
import hashlib
import unittest
import copy
from dataclasses import dataclass, field
import sympy
import networkx as nx
from pyvis.network import Network
from typing import *
from shapely.geometry import Polygon
from enum import StrEnum, Enum
from abc import ABC, abstractmethod

In [36]:
from itertools import product

class PatrolGrid:
    DIR = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    R = C = 0
    grid = []

    def in_bounds(self, r, c):
        return 0 <= r < self.R and 0 <= c < self.C
    
    def is_loop(self, r, c):
        vis = set()
        p = 0
        while self.in_bounds(r, c):
            if (r, c, p) in vis: return True
            vis.add((r, c, p))
            dr, dc = self.DIR[p]
            r += dr
            c += dc
            if self.in_bounds(r, c) and self.grid[r][c] == "#":
                r -= dr
                c -= dc
                p = (p + 1) % len(self.DIR)
        return False

    def calculate(self, filename):
        with open(filename, "r") as f:
            self.grid = [list(x) for x in f.read().splitlines()]
            self.R, self.C = len(self.grid), len(self.grid[0])
            vis = set()
            sr = sc = 0
            for r, c in product(range(self.R), range(self.C)):
                if self.grid[r][c] == "^":
                    sr, sc = r, c
                    break
            p = ans = 0
            while self.in_bounds(r, c):
                vis.add((r, c))
                dr, dc = self.DIR[p]
                r += dr
                c += dc
                if self.in_bounds(r, c) and self.grid[r][c] == "#":
                    r -= dr
                    c -= dc
                    p = (p + 1) % len(self.DIR)
            for r, c in vis:
                self.grid[r][c] = "#"
                ans += self.is_loop(sr, sc)
                self.grid[r][c] = "."
            print(ans)
PatrolGrid().calculate("big.txt")

1686


In [ ]:
def calculate(filename):
    def is_safe()
        pass
    def is_set(i, mask):
        return (mask >> i) & 1
    with open(filename, "r") as f:
        N = 4
        end_mask = (1 << N) - 1
        state = [0] * 4
        for i in [0, 2]:
            state[0] += 1 << i
        state[1] += 1 << 1
        state[2] += 1 << 3
        vis = set([tuple(state)])
        q = deque(tuple(state))
        steps = 0
        while q:
            n = len(q)
            for _ in range(q):
                state = q.popleft()
                if state[-1] == end_mask: 
                    print(steps)
                    return
                for lv in range(4):
                    items = []
                    for i in range(N):
                        if is_set(i, state[lv]): items.append(i)
                    for item in items:
                        if is_valid(item, lv + 1):
                            nstate = copy.deepcopy(state)
                        if is_valid(item, lv - 1):


            steps += 1
calculate("small.txt")
calculate("big.txt")

{(5, 2, 8, 0)}
[5, 2, 8, 0]
{(5, 2, 8, 0)}
[5, 2, 8, 0]


In [33]:
2 ** 10

1024

In [ ]:
from parse import compile

def calculate(filename):
    with open(filename, "r") as f:
        data = f.read().splitlines()
        row_translate = compile("rotate row y={:d} by {:d}")
        column_translate = compile("rotate column x={:d} by {:d}")
        rect = compile("rect {:d}x{:d}")
        ans = 0
        for line in data:
            print(line)
        print(ans)

# calculate("small.txt")
calculate("big.txt")

rect 1x1
rotate row y=0 by 20
rect 1x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 3
rect 2x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 3
rect 2x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 4
rect 2x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 3
rect 2x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 5
rect 1x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 6
rect 5x1
rotate row y=0 by 2
rect 1x3
rotate row y=2 by 8
rotate row y=0 by 8
rotate column x=0 by 1
rect 7x1
rotate row y=2 by 24
rotate row y=0 by 20
rotate column x=5 by 1
rotate column x=4 by 2
rotate column x=2 by 2
rotate column x=0 by 1
rect 7x1
rotate column x=34 by 2
rotate column x=22 by 1
rotate column x=15 by 1
rotate row y=2 by 18
rotate row y=0 by 12
rotate column x=8 by 2
rotate column x=7 by 1
rotate column x=5 by 2
rotate column x=2 by 1
rotate column x=0 by 1
rect 9x1
rotate row y=3 by 28
rotate row y=1 by 28
rotate row y=0 by 20
rotate col

In [ ]:
34, 1150